In [7]:
import pandas as pd 
import numpy as np

In [8]:
BASE_MODEL_FOLDER = '../results_ensemble'
MODEL_LIST = [
    'AE_SWA_large',
    'ALS_ensemble',
    'NCF_dist_exp_SWA',
    'SVDpp_ensemble_gaussian'
]

In [9]:
def create_val_matrix(split):
    tmp = []
    for m in MODEL_LIST:
        df = pd.read_csv(f'{BASE_MODEL_FOLDER}/{m}/{m}_split_{split}_val_results.csv')
        t = df['Prediction'].to_numpy()

        tmp.append(t)
    return np.column_stack(tmp)



In [10]:
def create_final_matrix():
    tmp = []
    for m in MODEL_LIST:
        df = pd.read_csv(f'{BASE_MODEL_FOLDER}/{m}/{m}_final_results.csv')
        t = df['Prediction'].to_numpy()

        tmp.append(t)
    return np.column_stack(tmp)



In [11]:
def rmse(x,y):
    return np.sqrt(np.mean((x-y)**2))

In [12]:
val_splits = []
for i in range(0, 5):
    val_splits.append(create_val_matrix(i))

In [13]:
val_truth = []
for i in range(0, 5):
    val_truth.append(pd.read_csv(f'../data_val_train_kfold/partition_{i}_val.csv')['Prediction'].to_numpy())

In [14]:
def combine_models(yhat, coeff):
    coeff = np.array(coeff)
    return np.matmul(yhat, coeff)

In [15]:
def save_predictions(res_path, predictions):
    test_pd = pd.read_csv('../data/sampleSubmission.csv')
    test_pd = test_pd.astype({'Prediction': 'float'})
    test_pd.iloc[:, 1] = predictions
    test_pd.to_csv(res_path, index=False, float_format='%.3f')

In [16]:
a = []
from sklearn.linear_model import Ridge
for i in range(0,5):
    coeffs = np.linalg.lstsq(val_splits[i], val_truth[i])[0]
    # ridge = Ridge(alpha=10)
    # ridge.fit(val_splits[i], val_truth[i])
    # coeffs = ridge.coef_
    a.append(coeffs)


coeffs = np.column_stack(a).mean(axis=1)

/tmp/ipykernel_178995/610613689.py:4: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coeffs = np.linalg.lstsq(val_splits[i], val_truth[i])[0]


In [17]:
coeffs

array([0.06188904, 0.27913845, 0.13833313, 0.52533407])

In [18]:

score = 0
for i in range(0,5):
    res = combine_models(val_splits[i], coeffs)
    score+= rmse(res, val_truth[i])
score/5

0.9709771320393543

In [19]:
coeffs

array([0.06188904, 0.27913845, 0.13833313, 0.52533407])

In [20]:
matrix_final = create_final_matrix()
res_final = combine_models(matrix_final, coeffs)

name = ''
for m in MODEL_LIST:
    name+=m
    name+='-'
name+='LSQ-results.csv'
save_predictions(name, res_final)